In [ ]:
import numpy as np
import math
import matplotlib.pyplot as plt
import os
import json
from datetime import datetime
import pandas as pd
from tqdm import tqdm
import bspline
import bspline.splinelab as splinelab
import sys
path = r'C:\Users\Yichen Jiang\Documents\PHD LIFE\Research\Hawkes Processes\Yelp'
sys.path.append(path)
from yelp_functions import *
from scipy.interpolate import splrep, BSpline

# Load R session

In [ ]:
%load_ext rpy2.ipython

# Import business name

In [ ]:
path = r'C:\Users\Yichen Jiang\Documents\PHD LIFE\Research\Hawkes Processes\Yelp'

path_business_withtext = os.path.join(path,'business with text features')

path_results = os.path.join(path,'results')

filenames = os.listdir(path_business_withtext)

results = os.listdir(path_results)

# OR choose business_id

In [ ]:
#business_id = '2GmGT-7QjowR1ihup3FbVA' # review number: 825
#business_id = '0FUtlsQrJI7LhqDPxLumEw' # review number: 2097
#business_id = '2weQS-RnoOBhb1KsHKyoSQ' # review number: 4534
#business_id = 'RESDUcs7fIiihp38-d6_6g' # review number: 8568
#business_id = 'P7pxQFqr7yBKMMI2J51udw' # review number: 3225
#business_id = '--9e1ONYQuAa-CB_Rrw7Tw' # review number: 1661
#business_id = '4JNXUYY8wbaaDmk3BPzlWw' # review number: 8570
#business_id = 'DkYS3arLOhA8si5uUEmHOw' # review number: 5206
#business_id = '5LNZ67Yw9RD6nf4_UhXOjw' # review number: 4522
#business_id = 'K7lWdNUhCbcnEvI0NhGewg' # review number: 6887
#business_id = 'cYwJA2A6I12KNkm2rtXd5g' # review number: 5575
#business_id = 'uW6UHfONAmm8QttPkbMewQ' # review number: 1463
#business_id = 'S-oLPRdhlyL5HAknBKTUcQ' # review number: 1139
#business_id = 'AV6weBrZFFBfRGCbcRGO4g' # review number: 4240
#business_id = 'ebTvBxSStI9Vf5Tpux_X3Q' # review number: 1107
business_id = 'J1RDyyPxhioqm8c_fi4P4Q' # review number: 1023

# Define other terms

In [ ]:
# list of deltas
list_deltas = [0.005, 0.05, 0.1, 1, 5]

# list of variables
list_variables = []
for i in range(0,5):
    list_variables.append(str(i+1)+'star')
list_variables += ['cool', 'funny', 'useful', 'average_stars', 'friend_count', 'elite_count', 'review_count', 'fan_count', 'yelping_since', 'mean_prob', 'mean_weight', 'sentiment_polarity', 'sentiment_subjectivity', 'text_length', 'textclean_length']

bspline_order = 3

knot_base = 'order' #'time' or 'order'

In [ ]:
count = 0
list_knot_base = ['order','time']
list_record = []

In [ ]:
results = os.listdir(path_results)
if 'time'and'order' in list_record:
    print(filenames[count].split('_with')[0]+ ' with all basis have been processed')
    count += 1
    list_record = []
if count <= len(filenames):
    for base in list_knot_base:
        if filenames[count].split('_with')[0]+'_'+str(base)+'.png' in results:
            print(filenames[count].split('_with')[0]+'_'+str(base)+'.png has already been processed')
            list_record.append(str(base))
        elif filenames[count].split('_with')[0]+'_'+str(base)+'.png' not in results:
            business_id = filenames[count].split('_with')[0]
            knot_base = base
            print('business_id is:',str(business_id),', knot_base is:',str(knot_base))
            break


In [ ]:
print(business_id)
print(knot_base)
print('the next count is:',count)

In [ ]:
for filename in filenames:
    print(filename.split('_with')[0])
    # update results
    results = os.listdir(path_results)
    if filename.endswith('.json'):
        if filename.split('_with')[0]+'_'+'order'+'_coefficients.csv' not in results:
            knot_base = 'order'
            business_id = filename.split('_with')[0]
            print(business_id, knot_base)
            try:
                main(business_id,path,list_deltas,list_variables,bspline_order,knot_base)
            except KeyError as error:
                print(error)
        elif filename.split('_with')[0]+'_'+'order'+'_coefficients.csv' in results:
            print(filename.split('_with')[0] + 'in order base has been finished')

        if filename.split('_with')[0]+'_'+'time'+'_coefficients.csv' not in results:
            knot_base = 'time'
            business_id = filename.split('_with')[0]
            print(business_id, knot_base)
            try:
                main(business_id,path,list_deltas,list_variables,bspline_order,knot_base)
            except KeyError as error:
                print(error)
        elif filename.split('_with')[0]+'_'+'time'+'_coefficients.csv' in results:
            print(filename.split('_with')[0] + 'in time base has been finished')    

In [ ]:
filename.split('_with')[0]+'_'+'order'+'_coefficients.csv' not in results

In [ ]:
filename.split('_with')[0]

In [ ]:
# create lookup dictionary and table
df_decaytable, dict_decaytable = create_decay_table(list_deltas)

# create dict_calendar for time-checking
dict_calendar = create_calendar()

# Define the main function

In [ ]:
def main(business_id,path,list_deltas,list_variables,bspline_order,knot_base):
    global dict_business
    global df_business
    global df_variables
    global df_bspline
    global dict_effective
    global dict_decayvalues
    global df_decaytable
    global dict_decaytable
    global dict_calendar
    global df_data
    global list_knot_order
    global list_knot_time
    
    # original business information (with user and text features)
    dict_business = {}
    path_user = os.path.join(path,'business','business with more than 500 reviews')
    with open(os.path.join(path_user,str(business_id)+'_with user information.json'),'r',encoding='utf-8') as file:
        for line in file.readlines():
            dict_business_user = json.loads(line)

    # save review information into dict_business_raw
    dict_business['RestaurantInfo'] = dict_business_user['RestaurantInfo']
    dict_business['Reviews'] = {}
    for review_id in dict_business_user['Reviews'].keys():
        dict_business['Reviews'][review_id] = {}
        for attribute in dict_business_user['Reviews'][review_id].keys():
            if attribute != 'user_info':
                dict_business['Reviews'][review_id][attribute] = dict_business_user['Reviews'][review_id][attribute]

    """ import user features"""
    get_user_features(business_id, path, dict_business)
    """ import text featrues"""
    get_text_features(business_id, path, dict_business)
    """ one-hot for star ratings"""
    one_hot_star(dict_business)

    """ str -> float or datetime"""
    for review_id in dict_business['Reviews']:
        for variable in dict_business['Reviews'][review_id].keys():
            if variable in list_variables:
                dict_business['Reviews'][review_id][variable] = float(dict_business['Reviews'][review_id][variable])
            elif variable == 'date':
                dict_business['Reviews'][review_id][variable] = datetime.strptime(dict_business['Reviews'][review_id][variable], "%Y-%m-%d %H:%M:%S")

    """ dictionary -> dataframe"""
    df_business = pd.DataFrame.from_dict(dict_business['Reviews']).T
    """ sort by datetime"""
    df_business = df_business.sort_values(by = 'date', axis = 0, ascending = True)

    """ add average_stars"""
    add_average_star(df_business)

    """ clear dataframe"""
    df_business = pd.concat([df_business['date'],df_business['stars'],df_business[list_variables]],axis=1)

    """ calculate decay values"""
    # dictionary for saving effective events (events with decay values > 1e-7)
    dict_effective = {}
    # dictionary for saving decay values for each event
    dict_decayvalues = {}

    calculate_decay_values(df_business, list_variables, list_deltas, dict_decaytable, dict_effective, dict_decayvalues)
    
    """ get dataframe """
    # get dataframe of independent variables and save it into .csv for running lasso model in R
    df_variables = pd.DataFrame(dict_decayvalues).T.drop('order',axis=1)

    """ b spline basis function """
    if knot_base == 'order':
        # get knot vector
        list_knot_order = get_knot_vector(df_business)
        # get b spline basis function
        df_business, df_bspline = b_spline_basis(bspline_order, list_knot_order, df_business)
    elif knot_base == 'time':
        # get knot vector
        list_knot_order, list_knot_time = get_knot_vector_time(df_business)
        # get b spline basis function
        df_business, df_bspline = b_spline_basis_time(bspline_order, list_knot_time, df_business)
    
    # export dataframe
    df_data = pd.concat([df_business['stars'],df_variables,df_bspline],axis = 1)
    df_data.to_csv(os.path.join(path,'dataframe_ready_to_use' , 'df_'+str(business_id)+'_'+str(knot_base)+'.csv'), index=False, quoting=1)
    # export business_id
    file = open(os.path.join(path,'dataframe_ready_to_use', 'business_id and knot_base.txt'),'r+')
    file.truncate()
    file.write(str(business_id)+','+str(knot_base)+'\n')
    file.close()   

# Run the main function

In [ ]:
print(business_id, knot_base)
main(business_id,path,list_deltas,list_variables,bspline_order,knot_base)

# Run R for lasso model

In [ ]:
%%R

##################################
##################################
###   Yelp_lasso regression    ###
##################################
##################################


# import library
library(glmnet)
library(stringr)

# setup working directory
setwd("C:/Users/Yichen Jiang/Documents/PHD LIFE/Research/Hawkes Processes/Yelp/dataframe_ready_to_use")
# read business_id and knot_base
parameters <- readLines("business_id and knot_base.txt")
business_id = unlist(strsplit(parameters,split = ','))[1]
knot_base = unlist(strsplit(parameters,split = ','))[2]
# import csv file
data_business = read.csv(paste('df_',business_id,'_',knot_base,'.csv',sep = ''))

list_variables = colnames(data_business)[which(str_detect(colnames(data_business),'decay')==TRUE)]
list_bspline = colnames(data_business)[which(str_detect(colnames(data_business),'spline')==TRUE)]

data_business_x = as.matrix(data_business[,c(list_variables,list_bspline)])
data_business_y = as.matrix(data_business['stars'])


# lasso model with penalty.factor
set.seed(2019)

# setup penalty factor
p.fac = c(rep(1,length(list_variables)),rep(0,length(list_bspline)))

lasso =cv.glmnet(data_business_x, data_business_y, penalty.factor = p.fac,alpha = 1, nfolds=5,intercept=FALSE)

# get coefficients
coef = as.matrix(coef(lasso,s="lambda.min"))
print(coef)

# get prediction
data_business_pred = predict(lasso, data_business_x)

# save prediction into data_business
data_business["prediction"] = data_business_pred 

# export data_business coefficients into csv
write.csv(data_business, file = paste('df_',business_id,'_',knot_base,'_with predictions.csv',sep = ''),row.names=FALSE)
write.csv(coef,file = paste('C:/Users/Yichen Jiang/Documents/PHD LIFE/Research/Hawkes Processes/Yelp/results/',business_id,'_',knot_base,'_coefficients.csv',sep = ''))


# Import data with predictions and draw plot in python

In [ ]:
"""
--- plot figure ---

"""
# import data with prediction values
df_data = pd.read_csv(os.path.join(path,'dataframe_ready_to_use' ,'df_'+str(business_id)+'_'+str(knot_base)+'_with predictions.csv'))
df_data.index = df_business.index

# create datafram to plot and add average star ratings
preds = pd.DataFrame({"preds":df_data['prediction'], "true":df_data['stars']}) 
preds["residuals"] = preds["true"] - preds["preds"]
preds["average"] = df_business['average_stars'] 

# Bspline for smoothing the true values (true star ratings)
length = len(preds)
index = np.arange(length)
y = preds['true']
list_year = [list_knot_order[i] for i in range(1,len(list_knot_order)-1)]
t,c,k = splrep(index,y,s=0,k=3,t=list_year)
prediction_bspline = BSpline(t, c, k, extrapolate=False)
index_bspline = np.linspace(index.min(), index.max(), 20000)

# plot
plt.style.use('ggplot')
fig = plt.figure(dpi = 80/2, figsize = (100/2, 50/2))
plt.title('Yelp: business_id = '+str(business_id)+', based on '+str(knot_base),  fontdict={'size':'50'})
plt.subplot(1, 1, 1)

plt.scatter(index, preds['preds'], label='Prediction', color='g',s=25) 
plt.plot(index_bspline,prediction_bspline(index_bspline), label='True star ratings with b spline function', color='k')
#plt.plot(prediction_bspline[0],prediction_bspline[1],label='Prediction by splev',color='k')
plt.plot(index, preds['average'], label = 'Average Star Ratings', color = 'r')

plt.xlabel('Review Order', fontdict={'size':'50'})
plt.ylabel('True Value, Predictions and Mean', fontdict={'size':'50'})
plt.xticks(fontsize = 50)
plt.yticks(fontsize = 50)
plt.axhline(y=preds['average'][len(preds)-1],color='r')
#plt.ylim(3.75,5)
#plt.xlim(0,1000)
plt.legend(loc='upper right',fontsize = 50)
plt.savefig(os.path.join(path,'results',str(business_id)+'_'+str(knot_base)+'.png'))
plt.show()    

In [ ]:
print(business_id)
print(knot_base)